---
toc: true
layout: post
title: Interview Function Implementation (Frontend)
description: Documenting step by step process when creating the interview function
type: tangibles
categories: [Java, Tri 2]
tags: [Markdown, Blog]
courses: { csa: {week: 20} }
---

My team's website for this trimester, Jinder, features an interview function that allows users to host video calls and join them. In addition, they will be able to see users who are currently online and set up meetings with interviewers/interviewees. 

This blog will focus on the frontend side of this feature. To learn more about the process for developing the backend for this feature, navigate to [Interview Function Implementation (Backend)](link)

First, let's make some basic HTML for our 'Interview Home' page, which is where users will navigate to in order to join/host meetings. 

```html
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Connected Users</title>
  <link rel="stylesheet" href="/css/styles.css">
  <link rel="stylesheet" href="/css/index.css">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
</head>
<body>
<div class="container">
  <div class="image-container">
    <img src="https://images.idgesg.net/images/article/2020/04/zoom_video_conferencing_online_meeting_remote_workers_one_user_connected_via_laptop_with_a_grid_of_twelve_participants_on_screen_2400x1600-100837446-large.jpg?auto=webp&quality=85,70" alt="Image">
  </div>

  <div class="main">
    <div class="new-meeting">
      <button id="newMeetingBtn">Create a New Meeting</button>
      <div class="join-meeting">
        <input type="text" placeholder="Meeting ID" id="meetingName">
        <button id="joinMeetingBtn">Join</button>
      </div>
    </div>
    <div class="connected-users">
      <button id="logoutBtn">Logout</button>
      <h2>Connected Users</h2>
      <ul id="userList"></ul>
    </div>
  </div>
</div>
```

With this, I've created a table to list connected users, a button to create meetings, a place to put Meeting IDs to join meetings, and a logout button. 

Before I put in the script, let me plan exactly what I want to happen with the website:
- Since users must be logged in for the feature to work correctly, I want the page to detect whether users are logged in or not and automatically redirect the user to the login page if they are not logged in
- Once users are logged in, their status is changed to "online." I want the "Connected Users" table to show all users that currently have the status "online"
- After creating a meeting, the website will automatically insert the username of whoever is creating the meeting, so it should be able to pull usernames from the backend
- The join button should be able to function similarly

Now, let's put in the javascript for the page:

```html
<script>
  function loadAndDisplayUsers() {
    // check if the user is connected
    const connectedUser = localStorage.getItem('connectedUser');
    if (!connectedUser) {
        window.location = 'login/';
        return;
    }
    const userListElement = document.getElementById("userList");
    // Clear any existing content in the userListElement
    userListElement.innerHTML = "Loading...";
    // Retrieve the userList from Local Storage
    fetch('http://localhost:8020/api/v1/users')
        .then((response) => {
            return response.json();
        })
        .then((data) => {
            console.log(data);
            displayUsers(data, userListElement);
        });
  }
  function displayUsers(userList, userListElement) {
      userListElement.innerHTML = "";
      // Loop through the userList and create list items to display each user
      userList.forEach(user => {
          const listItem = document.createElement("li");
          listItem.innerHTML = `
                  <div>
                      <i class="fa fa-user-circle"></i>
                      ${user.username} <i class="user-email">(${user.email})</i>
                  </div>
                  <i class="fa fa-lightbulb-o ${user.status === "online" ? "online" : "offline"}"></i>
              `;
          userListElement.appendChild(listItem);
      });
  }
  // Call the loadAndDisplayUsers function when the page loads
  window.addEventListener("load", loadAndDisplayUsers);
  function handleLogout() {
      fetch('http://localhost:8020/api/v1/users/logout', {
          method: 'POST',
          headers: {
              'Content-Type': 'application/json'
          },
          body: localStorage.getItem('connectedUser')
      })
          .then((response) => {
              return response;
          })
          .then((data) => {
              localStorage.removeItem('connectedUser');
              window.location.href = "login/";
          });
  }
  const logoutBtn = document.getElementById("logoutBtn");
  logoutBtn.addEventListener("click", handleLogout);
  function handleNewMeeting() {
    const connectedUserData = localStorage.getItem('connectedUser');
    if (!connectedUserData) {
        console.error('No connected user data found in local storage.');
        return;
    }
    try {
        const connectedUser = JSON.parse(connectedUserData);
        window.open(`videocall?username=${connectedUser.username}`, "_blank");
    } catch (error) {
        console.error('Error parsing connected user data:', error);
    }
  }
  // Attach the handleNewMeeting function to the "Create a New Meeting" button
  const newMeetingBtn = document.getElementById("newMeetingBtn");
  newMeetingBtn.addEventListener("click", handleNewMeeting);
  function handleJoinMeeting() {
      const roomId = document.getElementById("meetingName").value;
      const connectedUser = JSON.parse(localStorage.getItem('connectedUser'));
      const url = `videocall?roomID=${roomId}&username=${connectedUser.username}`;
      window.open(url, "_blank");
  }
  const joinMeetingBtn = document.getElementById("joinMeetingBtn");
  joinMeetingBtn.addEventListener("click", handleJoinMeeting);
</script>
```